# <span style="color:#00bfff;">Introducción a Pydantic: Validación de Datos en Python</span>

Este notebook es una guía práctica para entender y utilizar **Pydantic**, una de las librerías más importantes en el ecosistema de Python moderno para la validación de datos.

## <span style="color:#00bfff;">1. ¿Qué es Pydantic y cuándo utilizarlo?</span>

**Pydantic** es una librería que utiliza los *type hints* (indicadores de tipo) de Python para validar, serializar y parsear datos. En esencia, le permite definir la "forma" que deben tener sus datos a través de clases de Python.

#### Casos de uso comunes:
- **Desarrollo de APIs**: Es el motor de validación de frameworks como **FastAPI**. Asegura que los datos entrantes (requests) y salientes (responses) cumplan con un esquema definido.
- **Configuración de aplicaciones**: Permite cargar configuraciones desde archivos o variables de entorno, validando que todos los parámetros necesarios estén presentes y tengan el tipo correcto.
- **Procesamiento de datos estructurados**: Ideal para trabajar con formatos como JSON o YAML. Es especialmente útil para procesar las **salidas de Modelos de Lenguaje Grandes (LLM)**, que suelen devolver datos en formato JSON y necesitan ser validados rigurosamente.

In [ ]:
# Instalación (si no lo ha hecho antes)
%pip install pydantic pydantic[email]

Note: you may need to restart the kernel to use updated packages.


h:\Mi unidad\Docencia\Inteligencia Artificial - Ing Sistemas\2025-2\IA-Course-UdB\Unidad 1\fundamentos\.venv\Scripts\python.exe: No module named uv


## <span style="color:#00bfff;">2. Definir Modelos de Datos con `BaseModel`</span>

La clase fundamental en Pydantic es `BaseModel`. Se hereda de ella para crear modelos de datos personalizados. Los atributos de la clase se definen con sus respectivos *type hints*.

In [1]:
from pydantic import BaseModel, EmailStr, ValidationError
from typing import List, Optional

# Se define un modelo simple para un usuario
class Usuario(BaseModel):
    id: int
    nombre: str
    email: EmailStr  # Pydantic provee tipos especiales como EmailStr
    edad: Optional[int] = None # Campo opcional con un valor por defecto

## <span style="color:#00bfff;">3. Parsear, Validar y Serializar Datos</span>

Una vez definido el modelo, Pydantic se encarga del trabajo pesado: tomar datos "crudos" (como un diccionario), validarlos contra el modelo y crear una instancia de la clase si todo es correcto.

In [2]:
# Datos de entrada en formato de diccionario
datos_usuario = {
    "id": 123,
    "nombre": "Juan Pérez",
    "email": "juan.perez@email.com",
    "edad": 30
}

# 1. Parseo y Validación
try:
    usuario_validado = Usuario(**datos_usuario)
    print("✅ Datos validados correctamente:")
    print(usuario_validado)
except ValidationError as e:
    print(f"❌ Error de validación: {e}")

✅ Datos validados correctamente:
id=123 nombre='Juan Pérez' email='juan.perez@email.com' edad=30


In [3]:
# Ejemplo con datos incorrectos para provocar un error
datos_invalidos = {
    "id": "no-es-un-numero", # Tipo incorrecto
    "nombre": "Ana",
    # Falta el campo 'email', que es obligatorio
}

print("Intentando validar datos incorrectos:")
try:
    Usuario(**datos_invalidos)
except ValidationError as e:
    print(f"❌ Error de validación detectado:{e}")

Intentando validar datos incorrectos:
❌ Error de validación detectado:2 validation errors for Usuario
id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='no-es-un-numero', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing
email
  Field required [type=missing, input_value={'id': 'no-es-un-numero', 'nombre': 'Ana'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing


### Serialización de Datos

La serialización es el proceso inverso: convertir un objeto del modelo a un formato estándar como un diccionario o un string JSON. Pydantic lo hace muy sencillo con los métodos `.model_dump()` y `.model_dump_json()`.

In [4]:
# Usamos el objeto 'usuario_validado' que creamos antes

# Convertir a diccionario de Python
diccionario_usuario = usuario_validado.model_dump()
print("Objeto serializado a diccionario:")
print(diccionario_usuario)
print(f"Tipo: {type(diccionario_usuario)}\n")

# Convertir directamente a un string JSON
# El argumento 'indent=2' lo formatea para que sea legible
json_usuario = usuario_validado.model_dump_json(indent=2)
print("Objeto serializado a JSON:")
print(json_usuario)
print(f"Tipo: {type(json_usuario)}")

Objeto serializado a diccionario:
{'id': 123, 'nombre': 'Juan Pérez', 'email': 'juan.perez@email.com', 'edad': 30}
Tipo: <class 'dict'>

Objeto serializado a JSON:
{
  "id": 123,
  "nombre": "Juan Pérez",
  "email": "juan.perez@email.com",
  "edad": 30
}
Tipo: <class 'str'>


## <span style="color:#00bfff;">4. Trabajar con `json.loads()` y `json.dumps()`</span>

Aunque Pydantic ofrece métodos nativos para manejar JSON, es totalmente compatible con el módulo `json` de Python. El flujo habitual es:
1. Cargar un string JSON a un diccionario con `json.loads()`.
2. Validar ese diccionario con un modelo Pydantic.
3. Si se necesita un string JSON de vuelta, usar `json.dumps()` sobre el diccionario generado por `.model_dump()`.

In [5]:
import json

# 1. Partimos de un string JSON (podría venir de un archivo o una API)
json_string = '{"id": 456, "nombre": "Luisa Gomez", "email": "luisa.g@example.com"}'

# 2. Parseamos el string a un diccionario
datos_dict = json.loads(json_string)

# 3. Validamos el diccionario con nuestro modelo Pydantic
usuario_desde_json = Usuario(**datos_dict)

print("✅ Usuario validado desde un string JSON:")
print(usuario_desde_json)

✅ Usuario validado desde un string JSON:
id=456 nombre='Luisa Gomez' email='luisa.g@example.com' edad=None


## <span style="color:#00bfff;">5. Caso Práctico: Procesar la Salida de un LLM</span>

Imaginemos que le pedimos a un LLM que extraiga información de un texto y nos la devuelva en formato JSON. La salida del modelo puede ser inconsistente: puede incluir campos extra, tener tipos de datos como strings en lugar de números, etc. Pydantic es la herramienta perfecta para limpiar y validar esta salida.

In [6]:
# Modelo estricto para una receta de cocina
class Ingrediente(BaseModel):
    nombre: str
    cantidad: float
    unidad: str

class Receta(BaseModel):
    nombre_receta: str
    ingredientes: List[Ingrediente]
    tiempo_preparacion_min: int

# Salida simulada de un LLM (en formato string JSON)
# Notar que 'tiempo_preparacion_min' es un string y hay un campo extra 'dificultad'
salida_llm = '''
{
    "nombre_receta": "Torta de Chocolate",
    "ingredientes": [
        {"nombre": "Harina", "cantidad": 2.5, "unidad": "tazas"},
        {"nombre": "Azúcar", "cantidad": 2, "unidad": "tazas"},
        {"nombre": "Cacao en polvo", "cantidad": 0.75, "unidad": "tazas"}
    ],
    "tiempo_preparacion_min": "90",
    "dificultad": "media" 
}
'''

print("--- Salida cruda del LLM ---")
print(salida_llm)

# Proceso de validación:
# Pydantic automáticamente:
# 1. Convertirá 'tiempo_preparacion_min' de string a int.
# 2. Ignorará el campo extra 'dificultad' que no está en el modelo.
# 3. Validará que 'ingredientes' sea una lista de objetos que cumplen con el modelo 'Ingrediente'.

try:
    receta_validada = Receta.model_validate_json(salida_llm)
    print("\n--- ✅ Datos del LLM validados y estructurados ---")
    print(receta_validada)
    print("\nTipo de 'tiempo_preparacion_min':", type(receta_validada.tiempo_preparacion_min))
except ValidationError as e:
    print(f"\n--- ❌ Error al validar la salida del LLM ---\n{e}")

--- Salida cruda del LLM ---

{
    "nombre_receta": "Torta de Chocolate",
    "ingredientes": [
        {"nombre": "Harina", "cantidad": 2.5, "unidad": "tazas"},
        {"nombre": "Azúcar", "cantidad": 2, "unidad": "tazas"},
        {"nombre": "Cacao en polvo", "cantidad": 0.75, "unidad": "tazas"}
    ],
    "tiempo_preparacion_min": "90",
    "dificultad": "media" 
}


--- ✅ Datos del LLM validados y estructurados ---
nombre_receta='Torta de Chocolate' ingredientes=[Ingrediente(nombre='Harina', cantidad=2.5, unidad='tazas'), Ingrediente(nombre='Azúcar', cantidad=2.0, unidad='tazas'), Ingrediente(nombre='Cacao en polvo', cantidad=0.75, unidad='tazas')] tiempo_preparacion_min=90

Tipo de 'tiempo_preparacion_min': <class 'int'>


## <span style="color:#ff7800;">Conclusión</span>

Pydantic es una herramienta indispensable para escribir código Python robusto y fiable. Al definir explícitamente la estructura de sus datos, no solo previene errores, sino que también mejora la legibilidad del código y la experiencia de desarrollo gracias al autocompletado y la verificación de tipos de los editores modernos.